# IA Plugin analysis

## Import des bibliothèques nécessaires

In [2]:
from os.path import join
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, mean_absolute_error
from sklearn import preprocessing

## Récupération des fichiers

In [5]:
df_dataset = pd.read_csv(r"C:\Users\trist\Desktop\IA\dataset.csv", sep=";")

df_dataset.head()

,app_id,pge_name,vst_time,pge_estimated_time,pge_type_id
0,1,accueil,37483,16040,3
1,1,formulaire,37410,36712,4
2,1,accueil,14759,45610,3
3,1,article,25878,19356,4
4,1,article,41172,16620,2


## Jointures des fichiers

In [6]:
#pas besoin

## Préparation des données

In [7]:
#pas besoin

### Ajout de features

In [8]:
def calcul_pertinence(row):
    temps_estime = int(row.pge_estimated_time)
    temps_passe = int(row.vst_time)
    pourcentage_pertinence = {
        1 : [20, 30, 40],
        2 : [20, 30, 40],
        3 : [20, 30, 40],
        4 : [20, 30, 40]
    }

    ecart = (temps_passe - temps_estime) / temps_estime * 100
    if ecart < 0:
        ecart = ecart * -1

    for i in range(3):
        if ecart <= pourcentage_pertinence[row.pge_type_id][i]:
            return i+1
    return 3

df_dataset['pertinence_computed'] = df_dataset.apply(calcul_pertinence, axis=1)
df_dataset.head()

,app_id,pge_name,vst_time,pge_estimated_time,pge_type_id,pertinence_computed
0,1,accueil,37483,16040,3,3
1,1,formulaire,37410,36712,4,1
2,1,accueil,14759,45610,3,3
3,1,article,25878,19356,4,3
4,1,article,41172,16620,2,3


### Encodage des chaînes de caractères

In [9]:
#pas besoin car on ne traite pas de chaine de caractères

### Préparation des features

#### Suppression des chaînes de caractères et prise en charge des nouvelles features crée

In [10]:
features = [
    'app_id',
    'vst_time',
    'pge_estimated_time',
    'pge_type_id'
]
#On n'ajoute pas pertinence_computed car c'est ce que l'on cherche à prédire.
#J'ajoute pas la colonne pge_name car elle n'a pas d'intérêt pour l'apprentissage de l'IA

train_features = df_dataset[features] #X_train

label = ['pertinence_computed'] #ce que l'on cherche à prédire

train_label = df_dataset[label] #y_train

### Découpage du jeu d'entraînement en deux

In [11]:
train, test = train_test_split(df_dataset, test_size=0.3, random_state=42)

### Utilisation de l'algorithme RandomForest

In [12]:
rf = RandomForestClassifier(n_estimators=100,
                             max_depth=8,
                             random_state=0)

rf.fit(train_features, train_label) #apprentissage
predictions = rf.predict(test[features])

C:\Users\trist\anaconda3\envs\Maching_et_Deep_learning\lib\site-packages\ipykernel_launcher.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """


In [13]:
accuracy_score(test[label], predictions) 
#Comparaisons des données attendues avec les données prédits. 
#Une précision et ensuite donnée.

0.9386666666666666

#### Visualisation du nombre d'erreur

In [14]:
confusion_matrix(test[label], predictions)

array([[1152,    0,   17],
       [ 268,  141,   72],
       [   9,    2, 4339]], dtype=int64)

### Enregistrement des prédictions dans le fichier 'resultats_pertinence_page.csv'

In [15]:
test.head()

,app_id,pge_name,vst_time,pge_estimated_time,pge_type_id,pertinence_computed
10650,1,informatif,20941,26796,4,2
2041,1,formulaire,41257,20138,1,3
8668,1,formulaire,9725,27480,1,3
1114,1,informatif,4293,44925,3,3
13902,1,formulaire,33778,8640,2,3


# Simulation aprés récupération de données depuis la BDD

In [16]:
df_dataset_test = pd.read_csv("/Users/florentin/Documents/EPSI/I2B/WORKSHOP/IA/data/dataset/dataset_test_final.csv", sep=";")

df_dataset_test.head()
type(df_dataset_test)

FileNotFoundError: [Errno 2] File /Users/florentin/Documents/EPSI/I2B/WORKSHOP/IA/data/dataset/dataset_test_final.csv does not exist: '/Users/florentin/Documents/EPSI/I2B/WORKSHOP/IA/data/dataset/dataset_test_final.csv'

In [16]:
df_dataset_test['pertinence_computed'] = rf.predict(df_dataset_test[features])##en paramètre ca sera le jeu de données de la bdd

df_dataset_test.head()

,app_id,pge_name,vst_time,pge_estimated_time,pge_type_id,pertinence_computed
0,1,article,11810,47267,1,3
1,1,accueil,45020,10358,2,3
2,1,article,23193,43321,1,3
3,1,informatif,33252,42358,1,1
4,1,informatif,34253,13444,3,3


In [17]:
print(df_dataset_test[["pge_name","pertinence_computed"]])

tb = dict()

for index, row in test.head(2).iterrows():
    print(row['pge_name'], row['pertinence_computed'])
    tb[row['pge_name']] = row['pertinence_computed']

#formattage json
import json
json = json.dumps(tb)

print(json)
type(json)

         pge_name  pertinence_computed
0         article                    3
1         accueil                    3
2         article                    3
3      informatif                    1
4      informatif                    3
...           ...                  ...
19995     accueil                    1
19996     article                    3
19997     article                    3
19998  informatif                    3
19999  informatif                    3

[20000 rows x 2 columns]
informatif 2
formulaire 3
{'informatif': 2, 'formulaire': 3}
{"informatif": 2, "formulaire": 3}


str

# Test tranformation d'un array vers Dataframe

In [27]:
import numpy as np
np.random.seed(123)
e = np.random.normal(size=10)  
dataframe=pd.DataFrame(e, columns=['a']) 
type(e)
type(dataframe)
print (dataframe)

          a
0 -1.085631
1  0.997345
2  0.282978
3 -1.506295
4 -0.578600
5  1.651437
6 -2.426679
7 -0.428913
8  1.265936
9 -0.866740


# Test sauvegarde du model

In [30]:
import pickle
# save the model to disk
filename = 'finalized_model.sav'
pickle.dump(rf, open(filename, 'wb'))
 
# some time later...
 
# load the model from disk
loaded_model = pickle.load(open(filename, 'rb'))